In [ ]:
from platform import python_version
print(python_version())

### Calculating all possible Enrichment Analysis
  - for each LFC/FDR cutoff one calculates the Enrichment Analysis
  - We used Enricher python API
     - Reactome (2022)
     - Bioplanet (2019)
     - WikiPathways (2021 Human)
     - KEGG (2021 Human)
     - GO Biological Process (2021)
     - MSigDB Hallmark (2020)
   
### For each enriched pathways one calculates:
  - DEGs in the pathway
  - DEGs not in the pathway
  - index1, 2, 3, 4

### An index measures the trade-off between "LFC" and "Enriched Pathways" cufoff -> LFC - Enriched Pathway Trade-Off Method (LEPTOM)

  - We proposed and calculated the following possible indexes:

<p style="font-size: 20px; color: yellow;">
$index1=\sqrt{-log{_{10}}{FDR_{pathway}} * \frac{n}{N} }$ </p>

<p style="font-size: 20px; color: cyan;">
$index2=\sqrt{-log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} }$ </p>

<p style="font-size: 20px; color: orange;">
$index3=(-log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} * \frac{n}{N})^{1/3}$ </p>

<p style="font-size: 20px; color: pink;">
$index4=(abs\_LFC * -log{_{10}}{FDR_{LFC}} * -log{_{10}}{FDR_{pathway}} * \frac{n}{N})^{1/4}$ </p>

where,
  - n is the number of DEGs/DAPs found in the pathway
  - N is the total number of annotated DEGs/DAPs in the pathway (depend in the database, our default database is Reactome 2022)


#### Where
  - $FDR_{pathway}$ is the FDR from the enriched pathway
  - n - number of DEGs in the pathway
  - N - number of Genes annotated in the pathway by respective Dabase (Like Reactome, KEGG, etc)

In [ ]:
import os, sys, pickle

import numpy as np
import pandas as pd
pd.set_option('display.width', 100)
pd.set_option('max_colwidth', 80)
pd.set_option("display.precision", 3)

import yaml

import seaborn as sns
sns.set_context("notebook", font_scale=1.4)

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

sys.path.insert(1, '../src/')

from Basic import *
from enricher_lib import *

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

# !pip3 install pyyaml
with open('params.yml', 'r') as file:
    dic_yml=yaml.safe_load(file)

In [ ]:
root0=dic_yml['root0']
email=dic_yml['email']

project=dic_yml['project']
s_project=dic_yml['s_project']

gene_protein=dic_yml['gene_protein']
s_omics=dic_yml['s_omics']

has_age=dic_yml['has_age']
has_gender=dic_yml['has_gender']

want_normalized=dic_yml['want_normalized']

abs_lfc_cutoff_inf=dic_yml['abs_lfc_cutoff_inf']
s_pathw_enrichm_method=dic_yml['s_pathw_enrichm_method']
num_min_degs_for_ptw_enr=dic_yml['num_min_degs_for_ptw_enr']

tolerance_pathway_index=dic_yml['tolerance_pathway_index']
type_sat_ptw_index=dic_yml['type_sat_ptw_index']
saturation_lfc_index=dic_yml['saturation_lfc_index']
chosen_model_sampling=dic_yml['chosen_model_sampling']

case_list=dic_yml['case_list']

pval_pathway_cutoff=dic_yml['pval_pathway_cutoff']
fdr_pathway_cutoff=dic_yml['fdr_pathway_cutoff']
num_of_genes_cutoff=dic_yml['num_of_genes_cutoff']

run_list=dic_yml['run_list']
chosen_model_list=dic_yml['chosen_model_list']
i_dfp_list=dic_yml['i_dfp_list']

exp_normalization='quantile_norm' if want_normalized else None
normalization='not_normalized' if exp_normalization is None else exp_normalization

cfg=Config(project, s_project, case_list, root0)

case=case_list[0]

n_genes_annot_ptw, n_degs, n_degs_in_ptw, n_degs_not_in_ptw, degs_in_all_ratio=-1,-1,-1,-1,-1
abs_lfc_cutoff, fdr_lfc_cutoff, n_degs, n_degs_up, n_degs_dw=cfg.get_best_lfc_cutoff(case, 'not_normalized')


print(f"G/P LFC cutoffs: lfc={abs_lfc_cutoff:.3f}; fdr={fdr_lfc_cutoff:.3f}")
print(f"Pathway cutoffs: pval={pval_pathway_cutoff:.3f}; fdr={fdr_pathway_cutoff:.3f}; num of genes={num_of_genes_cutoff}")

In [ ]:
enr=enricheR(gene_protein, s_omics, project, s_project, root0,
             case_list, has_age, has_gender, clone_objects=False,
             exp_normalization=exp_normalization, geneset_num=0, 
             num_min_degs_for_ptw_enr=num_min_degs_for_ptw_enr, 
             tolerance_pathway_index=tolerance_pathway_index, 
             s_pathw_enrichm_method=s_pathw_enrichm_method,
             abs_lfc_cutoff_inf=abs_lfc_cutoff_inf, 
             type_sat_ptw_index=type_sat_ptw_index, saturation_lfc_index=saturation_lfc_index)

case=case_list[0]

enr.cfg.set_default_best_lfc_cutoff(normalization, abs_lfc_cutoff=1, fdr_lfc_cutoff=0.05)
ret, degs, degs_ensembl, dfdegs=enr.open_case(case, verbose=False)
print("\nEcho Parameters:")
enr.echo_parameters()
geneset_num=enr.geneset_num

In [ ]:
enr.case, enr.group, enr.gender, enr.age, enr.geneset_num, enr.abs_lfc_cutoff_inf

In [ ]:
enr.abs_lfc_cutoff_inf, abs_lfc_cutoff_inf

In [ ]:
for case in case_list:
    ret, degs, degs_ensembl, dfdegs = enr.open_case(case, save_file=True, verbose=False)
    enr.echo_parameters()
    print("\n\n\n")

### Summary of cases - below on can see the enriched tables for different databases

In [ ]:
geneset_num=0

for case in case_list:
    enr.open_case(case, force=False, verbose=False)
    enr.echo_parameters()
    print("")

### Cutoffs and Results

In [ ]:
for case in case_list:
    ret, degs, degs_ensembl, dfdegs=enr.open_case(case, verbose=False)

    print(f"For {case}")
    print(f"\tLFC cutoffs: lfc={enr.abs_lfc_cutoff:.3f}; fdr={enr.fdr_lfc_cutoff} #{enr.s_deg_dap}s={len(degs)}")
    print(f"\tPathway cutoffs: fdr={enr.pathway_fdr_cutoff:.3f}; num of genes={enr.num_of_genes_cutoff}, #Pathways={enr.n_pathways}, #{enr.s_deg_dap}s in pathwyas={enr.n_degs_in_pathways}\n")
    

In [ ]:
# enr.dflfc_ori.head(3)

In [ ]:
# df2=enr.dflfc_ori[ (enr.dflfc_ori.symbol == 'IGHA2') | (enr.dflfc_ori.symbol == 'A2M')]
# df2

In [ ]:
fname, fname_ori, title=enr.set_lfc_names()
fname, title

In [ ]:
enr.set_enrichment_name()

### Renaming files - if needed

In [ ]:
want_rename=False

if want_rename:
    root=enr.root_enrich
    _type='.tsv'
    _type=None
    pattern_src='not_normalized_'
    pattern_dst='not_normalized_cutoff_lfc_'
    
    rename_files(root, pattern_src, pattern_dst, _type=_type, verbose=False)

### Testing EnrichR API 

In [ ]:
case=case_list[6]
ret, degs, degs_ensembl, dfdegs=enr.open_case(case, verbose=False)
ret, len(degs), enr.n_degs

In [ ]:
enr.set_db(geneset_num=0)

In [ ]:
shortId, userListId=enr.open_session_upload_symbols(degs)
shortId, userListId

### All enriched cases for many databases

In [ ]:
fdr_ptw_cutoff_list=np.arange(0.05, 0.80, 0.05)
fdr_ptw_cutoff_list

In [ ]:
# dfsim=pdreadcsv( enr.cfg.fname_lfc_cutoff, enr.cfg.root_config)
dfsim=enr.cfg.open_all_lfc_cutoff()
try:
    print(len(dfsim))
except:
    dfsim=pd.DataFrame()

dfsim.tail(3)

### How many samples per case?

In [ ]:
for case in case_list:
    dfsim2=dfsim[ (dfsim.case == case) & (dfsim.normalization == enr.normalization) & 
                    (dfsim.n_degs >= enr.num_min_degs_for_ptw_enr)]
    print(f"case {case} #simulations {len(dfsim2)}")


In [ ]:
dfsim2=dfsim[ (dfsim.normalization == enr.normalization) & (dfsim.n_degs > 2)].copy()
dfsim2.index=np.arange(0, len(dfsim2))
print(len(dfsim2))

In [ ]:
for i in range(len(dfsim2)):
    row=dfsim2.iloc[i]

    degs=eval(row.degs)

    case=row.case
    abs_lfc_cutoff=row.abs_lfc_cutoff
    fdr_lfc_cutoff=row.fdr_lfc_cutoff

    print(i, case, abs_lfc_cutoff, fdr_lfc_cutoff, len(degs), degs[:10], '...')
    if i > 3: break

In [ ]:
dfsim[ (dfsim.case == 'g1_female') & (dfsim.abs_lfc_cutoff == 0.75) & (dfsim.fdr_lfc_cutoff == 0.65)]

In [ ]:
enr.fdr_ptw_cutoff_list

In [ ]:
enr.num_of_genes_list

### Calc all enrichment analyses

In [ ]:
geneset_num_list=[1, 2, 4, 5, 7]
geneset_num_list=[0, 1, 2, 4, 5, 7]
geneset_num_list=[0]

In [ ]:
enr.set_db(0, verbose=True)
enr.set_enrichment_name()

In [ ]:
enr.abs_lfc_cutoff_inf

### Testing g2a_male -> zero for 0.95, 0.05

In [ ]:
case='g2a_female'
ret, degs, degs_ensembl, dfqq=enr.open_case(case)
len(degs), enr.n_degs

In [ ]:
enr.abs_lfc_cutoff=0.95
enr.fdr_lfc_cutoff=0.05
degs2, _, _=enr.list_of_degs()
len(degs2)

In [ ]:
abs_lfc_cutoff=0.5
fdr_lfc_cutoff=0.3
degs2, _, _=enr.list_of_degs_set_params(abs_lfc_cutoff, fdr_lfc_cutoff, verbose=True)
len(degs2)

### Calc DEFAULT paramenters Enrichment Analysis

In [ ]:
force=False
verbose=False
geneset_num_list=[0, 1, 2, 4, 5, 7]
enr.calc_default_enrichment_analysis(geneset_num_list=geneset_num_list, force=force, verbose=verbose)

### Reactome in Enricher

In [ ]:
enr.abs_lfc_cutoff_inf

In [ ]:
df_fdr=enr.open_fdr_lfc_correlation(case, enr.abs_lfc_cutoff_inf)
df_fdr.head(3)

In [ ]:
dfsim=enr.open_simulation_table()
dfsim.tail(3)

In [ ]:
dfsim.case.unique()

In [ ]:
force=False
verbose=False
geneset_num_list=[0]
# remove the comments - it last some minutes
enr.calc_all_enrichment_analysis(geneset_num_list, force=force, verbose=verbose)

In [ ]:
force=False
verbose=False
geneset_num_list=[1, 2, 4, 5, 7]
# remove the comments - it last some minutes
enr.calc_all_enrichment_analysis(geneset_num_list, force=force, verbose=verbose)

### Sampling Pathways 

In [ ]:
dfa=enr.count_sampling(geneset_num_list=[0], prompt_verbose=True)
len(dfa)

In [ ]:
fig, dfa=enr.barplot_sampling_cutoffs(prompt_verbose=False, verbose=False)
fig.show()

### Other tests

In [ ]:
force=False; verbose=False
num_min_degs_for_ptw_enr=3

geneset_num_list=[1, 2, 4, 5, 7]
geneset_num_list=[0, 1, 2, 4, 5, 7]
geneset_num_list=[0]

want_test=False

if want_test:
    icount=-1
    for case in case_list:
        if not enr.open_case_simple(case):
            print(f"Problems for {case} !!!!")
            continue
        
        dfsim2=dfsim[ (dfsim.normalization == enr.normalization) & (dfsim.case == case) &
                        (dfsim.n_degs >= num_min_degs_for_ptw_enr)]
        
        for i in range(len(dfsim2)):
            icount += 1
            
            row=dfsim2.iloc[i]
    
            degs=eval(row.degs)
            case=row.case
            
            abs_lfc_cutoff=row.abs_lfc_cutoff
            fdr_lfc_cutoff=row.fdr_lfc_cutoff
    
            degs2, _=enr.list_of_degs_params(abs_lfc_cutoff, fdr_lfc_cutoff, verbose=False)
    
            if len(degs) != len(degs2):
                print("Error:", case, abs_lfc_cutoff, fdr_lfc_cutoff, len(degs), len(degs2))
                continue
    
            # if i > 10:break
            enr.calc_EA_dataset_symbol(degs, return_value=True, force=force, verbose=verbose)
            if icount%100==0:
                print(case, len(degs), abs_lfc_cutoff, fdr_lfc_cutoff)
                enr.echo_degs()
                print("")
                enr.echo_enriched_pathways()
                print("\n")


### Reactome, Bioplanet, KEGG

In [ ]:
enr.dbs_list

In [ ]:
[enr.dbs_list[i] for i in [0, 1, 2, 4, 5, 7]]

In [ ]:
enr.set_which_db('xxx')

In [ ]:
enr.set_which_db('Reactome_2022')

In [ ]:
enr.set_which_db('Reactome')

In [ ]:
enr.set_which_db('reactome')

In [ ]:
enr.set_which_db('KEGG_2021')

In [ ]:
enr.set_which_db('KEGG')

In [ ]:
enr.set_db(geneset_num=0)

### Start with Reactome

In [ ]:
case_list

In [ ]:
case=case_list[0]
enr.open_case(case, verbose=False)
degs, dfdegs=enr.list_of_degs(force=False, verbose=True)

# enr.set_which_db('Reactome_2022')
geneset_num=0
force=False; verbose=False

print(f"case: {case}")
print(f"there are {len(degs)} for fdr < {enr.fdr_lfc_cutoff:.3f} and lfc >= {enr.abs_lfc_cutoff:.3f}")
print(f"Pathway filter fdr < {enr.fdr_pathway_cutoff:.3f} and lfc >= {enr.pval_pathway_cutoff:.3f} and num_of_genes_cutoff={enr.num_of_genes_cutoff}")

enr.calc_EA_dataset_symbol(degs, force=force, verbose=verbose)

In [ ]:
i=0
case=case_list[i]
ret, degs, dfdegs=enr.open_case(case, verbose=False)

enr.df_enr

In [ ]:
print(enr.geneset_lib)

row=enr.get_enriched_pathway_line(i_line=0)

if row is not None:
    print(row.pathway, row.pathway_id)
    print(len(enr.genes_in_pathway), enr.genes_in_pathway)


### Enriched DEGs

In [ ]:
len(enr.all_enr_degs), ",".join(enr.all_enr_degs)

In [ ]:
",".join(enr.degs)

In [ ]:
enr_found_degs=[x for x in enr.degs if x in enr.all_enr_degs]
",".join(enr_found_degs)

### Found genes 

In [ ]:
len(enr.all_enr_degs), ",".join(enr.all_enr_degs)

In [ ]:
len(enr.enr_found_degs), ",".join(enr.enr_found_degs)

### Not found genes in pathways

In [ ]:
len(enr.enr_not_found_degs), ",".join(enr.enr_not_found_degs)

### BioPlanet

In [ ]:
enr.set_db(geneset_num=4)

In [ ]:
dfbiop=enr.open_db_pathway()
print(len(dfbiop))
dfbiop.head(3)

### Bioplanet diseases

In [ ]:
_=enr.open_bioplanet_disease(force=False)
dfdisease=enr.dfdisease
print(len(dfdisease))
dfdisease.head()

### Bioplanet Categories

In [ ]:
 _=enr.open_bioplanet_category(force=False)
dfbiop_cat=enr.dfbiop_cat
print(type(dfbiop_cat.category))
print(len(dfbiop_cat))
dfbiop_cat.head()

In [ ]:
def all_equal_list(cols1, cols2):
    if cols1 is None and cols2 is None: return True
    if cols1 == [] and cols2 == []: return True
    
    if len(cols1) != len(cols2): return False
    
    soma=np.sum([1 if cols1[i] != cols2[i] else 0 for i in range(len(cols1))])
    return soma == 0

cols1=list(enr.dflfc_all.columns)

cols2=['probe', 'symbol', 'geneid', 'description', 'logFC', 'meanExpr', \
        't.stat', 'p-value', 'fdr', 'B', 'chr.range', 'org.chromosome', \
        'forward.reverse', 'nuc.sequence', 'gemmaid', 'go.term']

all_equal_list(cols1, cols2)

In [ ]:
all_genes=[]
for i in range(len(dfsig)):
    genes=eval(dfsig.iloc[i].genes)
    # print(i, len(genes))
    all_genes += genes
    
all_genes=np.unique(all_genes)
all_genes.sort()
all_genes

not_found_genes=np.array([x for x in enr.deg_list if not x in all_genes])
not_found_genes

In [ ]:
def find_hugo_symbol(gene):
    try:
        i=enr.gene.dic_genes[gene]
        if isinstance(i, list):
            i=i[0]
            
        mat=enr.gene.df_synonyms.iloc[i]['synonyms']
        print(">>>", gene, i, mat, type(mat))
        if isinstance(mat, str):
            mat=eval(mat)
            
        gene0=mat[0]
    except:
        gene0=gene

    return gene0

In [ ]:
gene='SEPP1'
find_hugo_symbol(gene)

In [ ]:
enr.dic_genes[gene]

In [ ]:
lista=[x for x in os.listdir(enr.root_result) if 'taubate_PAC_UP_' in x and not '~lock' in x ]
print(len(lista))
", ".join(lista)


In [ ]:
root=enr.root_result
_type='.tsv'
_type=None
pattern_src='taubate_PAC'
pattern_dst='taubate_MAP'

if _type is None:
    lista=[x for x in os.listdir(root) if pattern_src in x and not '~lock' in x ]
else:
    lista=[x for x in os.listdir(root) if pattern_src in x and not '~lock' in x and x.endswith(_type)]

if lista == []:
    print(f"There are no files in {root}")

for fname in lista:
    fname_src=os.path.join(root, fname)

    fname_dst=fname.replace(pattern_src, pattern_dst)
    fname_dst=os.path.join(root, fname_dst)

    if not  os.path.exists(fname_src):
        print(f"fname does not exist: '{fname_src}'", fname_src)
        continue
    if os.path.exists(fname_dst):
        print(f"fname already exists: '{fname_dst}'")
        continue
        
    if fname_dst == fname_src:
        print(f"fname did not change: 'fname_src'")
        continue
        
    print(fname_src, 'to', fname_dst)
    os.rename(fname_src, fname_dst)

In [ ]:

def open_enriched_table_manually(self, fname, root):
    enr.df_enr0, enr.df_enr=None, None
    fname_enr, fname_enr_cutoff=bpx.set_enrichment_name()

    bpx.fname_enr=fname_enr
    bpx.fname_enr_cutoff=fname_enr_cutoff
    bpx.root_enr=root

    filefull=os.path.join(root, fname_enr)

    if not os.path.exists(filefull):
        bpx.deg_list=[]
        print("File does not exists '%s'"%(filefull))
        return None

    df_enr0=pdreadcsv(fname_enr, bpx.root_enr)
    ret=False if df_enr0 is None or df_enr0.empty else True

    if ret:
        df_enr0.columns=bpx.old_pathway_cols
        df_enr0=df_enr0[bpx.sel_pathway_cols]

        df_enr=df_enr0[ (df_enr0.fdr_pathway_cutoff < 0.05) & (df_enr0.num_of_genes >= bpx.num_of_genes_cutoff)]

        bpx.df_enr0=df_enr0
        bpx.df_enr=df_enr

        print(f">>> df_enr has {len(df_enr)} enrichment pathways and max(fdr_pathway_cutoff)={df_enr.fdr_pathway_cutoff.max():%.3f}")
        all_enr_genes=[]
        for i in range(len(df_enr)):
            s_genes=df_enr.iloc[i].genes
            if isinstance(s_genes, str) and s_genes != '':
                genes=list(s_genes.split(';'))
                all_enr_genes += genes

        all_enr_genes=list(np.unique(all_enr_genes))
    else:
        bpx.df_enr0=None
        bpx.df_enr=None
        all_enr_genes=[]

    bpx.all_enr_genes=all_enr_genes
    return ret

### Studied symbos in pubmed

In [ ]:
prefix="taubate_covid19"
inidate="2019/10/01"
enddate="2030/12/31"

email="flalix@gmail.com"

sleep_entrez=[30, 90, 300]; retmax=100000,

nlp=NLP(email, prefix, root0, 
          sleep_entrez=[30, 90, 300], retmax=100000,
          only_title_abstract=True, text_quote='',
          inidate=inidate, enddate=enddate, root_colab=root_colab, 
          force_query=False, verbose_query=False, dec_ncpus=2)

df_my_gene, df_my_gene_syn=nlp.gene.open_my_gene()
print(df_my_gene.shape)
df_my_gene.head(3)

In [ ]:
dfsymb_perc=nlp.find_symbols_build_perc_table(force=False)
print(len(dfsymb_perc))
dfsymb_perc.head(3)

In [ ]:
dfsymb_perc[dfsymb_perc.symbol == 'VWF']

In [ ]:
all_pubmed=list(dfsymb_perc.symbol)
print(len(all_pubmed))
all_pubmed.sort()

degs_notin=[x for x in degs if not x in all_pubmed]
degs_notin.sort()

print(len(degs_notin))
", ".join(degs_notin)

In [ ]:
dfsymb_perc[dfsymb_perc.symbol == 'CLEC3B']

In [ ]:
not 'VWF' in all_pubmed

In [ ]:
all_pubmed=list(dfsymb_perc.symbol)
print(len(all_pubmed))
all_pubmed.sort()

# ", ".join(all_pubmed)

In [ ]:
geneset_num_list=[0, 1, 2, 4, 5, 7]
prompt_verbose=False

dic={}; i=-1
for geneset_num in geneset_num_list:
    enr.set_db(geneset_num, verbose=True)

    s_start=f"enricheR_{enr.geneset_lib}"

    for case in case_list:
        files=[x for x in os.listdir(enr.root_enrichment) if x.startswith(s_start) and case in x]
        if prompt_verbose: print("\tcase", case, len(files))

        i+=1
        dic[i]={}
        dic2=dic[i]
    
        dic2['geneset_num']=enr.geneset_num
        dic2['geneset_lib']=enr.geneset_lib
        dic2['case']=case
        dic2['n']=len(files)

    if prompt_verbose: print('')

dfa=pd.DataFrame(dic).T
dfa.head(3)

In [ ]:
title=f'Sampling cutoffs per geneset'
yaxis_title=f'number of samples'
width=1100
height=700

colors=['green', 'red', 'blue', 'brown', 'yellow', 'cyan', 'lightgreen', 'pink', 'gray', 'lightblue']

geneset_lista=dfa.geneset_lib.unique()
colors_geneset=colors[:len(geneset_lista)]

_n_rows=int(np.ceil(len(enr.case_list)/2))
fig=make_subplots(rows=_n_rows, cols=2, subplot_titles=enr.case_list)

nrow=1
ncol=0

for case in enr.case_list:
    
    dfa2=dfa[dfa.case == case].copy()
    dfa2=dfa2.sort_values('geneset_lib')
    dfa2.index=np.arange(0, len(dfa2))

    ncol += 1
    if ncol == 3:
        ncol=1
        nrow += 1
        
    
    fig.add_trace(go.Bar(x=dfa2.geneset_lib, y=dfa2.n, marker_color=colors_geneset, name=case), row=nrow, col=ncol)

fig.update_layout(
            autosize=True,
            title=title,
            width=width,
            height=height*_n_rows,
            plot_bgcolor='lightgray',
            xaxis_title="cases",
            yaxis_title=yaxis_title,
            showlegend=False,
            font=dict(
                family="Arial",
                size=14,
                color="Black"
            )
)

figname=title_replace(title)
figname=os.path.join(enr.root_figure, figname+'.html')

fig.write_html(figname)
if verbose: print(">>> HTML and png saved:", figname)
fig.write_image(figname.replace('.html', '.png'))

fig.show()

In [ ]:
dfa2